In [42]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('ggplot')
%matplotlib inline
from tqdm import tqdm
from multitest import MultiTest
from datasets import load_dataset

In [43]:
from scipy.interpolate import interp1d
import pandas as pd

In [73]:
import numpy as np
import torch
import os
dataset = load_dataset("potsawee/wiki_bio_gpt3_hallucination")
#dataset = load_dataset("aadityaubhat/GPT-wiki-intro")

Using custom data configuration potsawee--wiki_bio_gpt3_hallucination-e8e8a8d1fbf50281
Found cached dataset json (/Users/kipnisal/.cache/huggingface/datasets/potsawee___json/potsawee--wiki_bio_gpt3_hallucination-e8e8a8d1fbf50281/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

# To do:
- Use GPT3wikibio dataset to generate mixed pages of GPT and human text
-

In [74]:
ll0 = []
ll1 = []
for d in dataset['train']:
    ll0.append(len(d['gpt3_sentences']))
    ll1.append(len(list(nlp(d['wiki_bio_text']).sents)))


In [67]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [95]:

def mix_lists(lst0: list, lst1: list, eps: float) -> dict:
    """
    Mix elements from lst0 and lst1 according to the mixture probability eps

    Params:
        :lst0   the base list
        :lst1   the mixture list
        :eps    the probability of replacing an element in lst0 by the corresponding element in lst1

    Returns:
        :mixted list
        :I   indexes of replaced elements
    """
    n = min(len(lst1), len(lst0))
    I = np.random.rand(n) < eps
    return [lst1[i] if i in I else lst0[i] for i in range(n)], I


In [130]:
def mix_documents(text0, text1, eps):
    sents0 = list(nlp(text0).sents)
    sents1 = list(nlp(text1).sents)
    sents01, I = mix_lists(sents0, sents1, eps=eps)
    return pd.DataFrame({'text': [s.text for s in sents01], 'mix': I})

In [135]:
eps = 0.2
for d in dataset['train']:
    df = mix_documents(d['gpt3_text'], d['wiki_bio_text'], eps=0.1 )
    df.to_csv(f"Data/GPT3/mixtures/{d['wiki_bio_test_idx']}_mix_eps{eps}.csv")

{'gpt3_text': "Ronald William George Barker, OBE (25 September 1929 – 3 October 2005), was an English actor, comedian, and writer. He was known for roles in British comedy television series such as Porridge, The Two Ronnies, and Open All Hours.\n\nBarker's acting career began in repertory theatre and, in the early 1960s, he appeared in several episodes of the BBC's police series Z-Cars. He also had minor roles in films such as The Magnificent Seven (1960) and The Wrong Arm of the Law (1963). In 1965, he was cast in the lead role of the BBC's sitcom The Frost Report, which also featured John Cleese, Ronnie Corbett, and other future stars of British comedy.\n\nBarker's career reached its peak in the 1970s with the BBC sketch show The Two Ronnies, which he co-wrote and performed with Ronnie Corbett. The show ran for 16 years and earned Barker two BAFTA awards. He also starred in the sitcoms Porridge (1974–1977) and Open All Hours (1973–1985).\n\nB",
 'wiki_bio_text': 'Ronald William Georg

In [121]:
pd.DataFrame({0:sents0, 1:sents1[:len(sents0)] })

,0,1
0,"(Ronald, William, George, Barker, ,, OBE, (, 2...","(Ronald, William, George, "", Ronnie, "", Barker..."
1,"(He, was, known, for, roles, in, British, come...","(He, was, known, for, roles, in, British, come..."
2,"(Barker, 's, acting, career, began, in, repert...","(Barker, began, acting, in, repertory, theatre..."
3,"(He, also, had, minor, roles, in, films, such,...","(He, had, his, first, success, at, the, Oxford..."
4,"(In, 1965, ,, he, was, cast, in, the, lead, ro...","(During, this, period, ,, he, was, in, the, ca..."
5,"(Barker, 's, career, reached, its, peak, in, t...","(He, got, his, television, break, with, the, s..."
6,"(The, show, ran, for, 16, years, and, earned, ...","(He, joined, David, Frost, 's, production, com..."
7,"(He, also, starred, in, the, sitcoms, Porridge...","(After, rejoining, the, BBC, ,, Barker, found,..."
8,(B),"(He, starred, in, the, sitcoms, "", Porridge, ""..."


In [122]:
I

array([False,  True, False, False, False, False, False, False, False])

In [107]:
pd.DataFrame({'sr':I, 'text' : sents01})

,sr,text
0,False,"(Ronald, William, George, "", Ronnie, "", Barker..."
1,True,"(He, was, known, for, roles, in, British, come..."
2,True,"(Barker, 's, acting, career, began, in, repert..."
3,False,"(He, also, had, minor, roles, in, films, such,..."
4,False,"(In, 1965, ,, he, was, cast, in, the, lead, ro..."
5,False,"(Barker, 's, career, reached, its, peak, in, t..."
6,True,"(The, show, ran, for, 16, years, and, earned, ..."
7,False,"(He, also, starred, in, the, sitcoms, Porridge..."
8,False,(B)


array([False, False, False, False,  True, False, False, False, False])